In [1]:
import pandas as pd
import re
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

In [33]:
def syn_gen(col_name):
    if col_name == 'nan':
        return ['nan']
    col_name = re.sub('\(.*\)','',col_name)
    col_name = col_name.lower()
    name = col_name.split(',')[0]
    names = []
    names.append(name)                                           # As given
    
    name = re.subn('of|\sand|&', '', name, flags = re.IGNORECASE)[0]
    
    name = re.subn('\s+', ' ', name)[0]                          # Removing 'of' and 'and'
    names.append(name)
    names.append(''.join([word[0] for word in name.split()]).upper())
    
    if "'" in col_name:    
        names.append(''.join([word[0] for word in filter(None,re.split('\W+', name))]).upper())
    if col_name.count('.') > 1:
        names.append(''.join(re.split(r'\.\s?', col_name)[:-1]).upper())
        
        try:
            pattern = re.compile('\s?univ\S*\s?|\s?coll\S*\s?|\s?inst\S*\s?|\sres\S*\sfound\S*\s?')
            inst_type = pattern.findall(name)[0]
            names.append(names[-1]+inst_type)
        except:
            pass
        
    if ',' in col_name:
        location_full = [name+re.findall(',.*', col_name)[0][1:] for name in names]
        try:
            location_short = [name+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)]
            location_short.extend([name+'-'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+'/'+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            location_short.extend([name+' '+col_name.split(',')[-1].split()[0][0].upper() for name in filter(lambda x: x.isupper(), names)])
            names.extend(location_short)
        except:
            pass
        names.extend(location_full)
    return list(filter(None,set(names)))

syn_gen("T.M. Bhagalpur University, Bhagalpur")

['TM/B',
 'TMB',
 'TM-B',
 'TBU/B',
 'TBU B',
 'TM bhagalpur',
 'TBU',
 'TM B',
 'TM university',
 'TBU-B',
 'TBU bhagalpur',
 'TM',
 'TM university bhagalpur',
 'TBUB',
 't.m. bhagalpur university bhagalpur',
 't.m. bhagalpur university']

In [3]:
data = pd.read_csv('../all_institutions.csv').astype(str)
# ddata = dd.from_pandas(data['name'], npartitions=2)
# data['col_synonyms'] = ddata.map_partitions(lambda df: df.apply(syn_gen), meta=pd.DataFrame).compute(get=get)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data.to_csv('../col_synonym_data.csv')

In [ ]:
data['col_name'] = data['name'].apply(syn_gen)

In [ ]:
entity = 'name'
data = pd.read_csv('../all_colleges_mereexams.csv')#.loc[:,entity]
data[data['name'].isin(data['name'][data['name'].duplicated()])].sort_values("name")# data.shape

In [19]:
def rm_brk(s):
    return re.sub('\(.*\)','',str(s))
clgs = pd.DataFrame(data.name)

In [34]:
univ_name_synonyms = []
univ_names = list(data['Affiliated_to_University'].unique())
for name in tqdm(univ_names):
    univ_name_synonyms.append([name]+syn_gen(name))

100%|██████████| 299/299 [00:00<00:00, 33117.59it/s]


In [35]:
len(list(data['Affiliated_to_University'].unique()))

299

In [42]:
names = pd.DataFrame(univ_name_synonyms)

In [43]:
names.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,"Sankalchand Patel University, Visnagar",SPUV,SPU visnagar,SPU/V,SPU,SPU V,SPU-V,sankalchand patel university,sankalchand patel university visnagar,None,None,None,None,None,None,None,None,None,None
1,"Rani Channamma University, Belagavi",rani channamma university,rani channamma university belagavi,RCU/B,RCU,RCU belagavi,RCU B,RCU-B,RCUB,None,None,None,None,None,None,None,None,None,None
2,"Saurashtra University, Rajkot",SU rajkot,SU/R,SU-R,SUR,SU R,SU,saurashtra university,saurashtra university rajkot,None,None,None,None,None,None,None,None,None,None
3,BHAKTA KAVI NARSINH MEHTA UNIVERSITY,bhakta kavi narsinh mehta university,BKNMU,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"R K University, Rajkot",RKU rajkot,r k university,RKU-R,RKUR,RKU,RKU R,r k university rajkot,RKU/R,None,None,None,None,None,None,None,None,None,None


In [52]:
import csv
names.to_csv('univ_names.csv', index = None, quoting=csv.QUOTE_NONE, escapechar=' ')